In [1]:
import os
import numpy as np
import pandas as pd
import scanpy as sc

In [2]:
sc.set_figure_params(dpi=100)
import re
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
out_dir = "output"
os.makedirs(out_dir, exist_ok=True)

In [4]:
adata_old= sc.read_h5ad("/data/projects/robin/segmentation/classify_nuc/sn_annotated_210325.h5ad")
adata_old.obs['Disease']= np.where(adata_old.obs['sample'].str.contains('CN'), 'Cntrl', 'ANCA')

In [5]:
adata= sc.read_h5ad("./output/NucSeq_integrated_v1.h5ad")
adata.obs['Disease']= np.where(adata.obs['sample'].str.contains('CN'), 'Cntrl', 'ANCA')

In [6]:
#adata is the new object and adata_old is the old one.
for col in ["celltype_l1_codes", "celltype_l1_prob", "celltype_l1"]:
    adata.obs[col] = adata_old.obs[col].tolist()

In [7]:
adata.obs.head()

,sample,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,pct_counts_in_top_200_genes,pct_counts_in_top_500_genes,total_counts_mt,...,pct_counts_ribo,total_counts_hb,log1p_total_counts_hb,pct_counts_hb,n_genes,leiden,Disease,celltype_l1_codes,celltype_l1_prob,celltype_l1
AAACGCTCACGCGCTA-1,nCN017K,3905,8.270269,11472.0,9.347752,20.231869,26.847978,35.887378,51.664923,526.0,...,0.252789,3.0,1.386294,0.026151,3905,7,Cntrl,1.000000,1.000000,IC
AAAGAACAGCACTCTA-1,nCN017K,2105,7.652546,4236.0,8.351611,21.813031,28.918791,38.456091,56.043437,335.0,...,0.306893,0.0,0.000000,0.000000,2105,2,Cntrl,0.994376,0.994376,DTL
AAAGGATAGCATTGTC-1,nCN017K,1729,7.455877,4007.0,8.296047,35.687547,42.924881,52.083853,68.380334,310.0,...,0.299476,1.0,0.693147,0.024956,1729,10,Cntrl,1.000000,1.000000,podocyte
AAAGGGCAGAATCGAT-1,nCN017K,2692,7.898411,7303.0,8.896177,23.798439,31.671916,41.859510,59.304395,149.0,...,0.205395,0.0,0.000000,0.000000,2692,7,Cntrl,1.000000,1.000000,IC
AAAGGGCCACCTCAGG-1,nCN017K,1097,7.001246,1653.0,7.410952,26.134301,32.788869,44.888082,63.883848,119.0,...,0.362976,1.0,0.693147,0.060496,1097,3,Cntrl,0.995571,0.995571,CNT


In [8]:
adata.obs.celltype_l1.unique()

array(['IC', 'DTL', 'podocyte', 'CNT', 'PT', 'PC',
       'parietal epithelial cell', 'T', 'TAL', 'DCT', 'EC',
       'endothelial cell', 'ncMON', 'B', 'mesangial cell', 'PL', 'ATL',
       'NEU', 'FIB', 'cycNKC/T', 'MDC', 'cDC', 'NKC/T', 'VSM/P', 'MAST',
       'MAC-M2', 'N', 'pDC', 'cycMNP'], dtype=object)

In [9]:
# Merge cell types : "EC" and "endothelial cell"
rename_dict = {
    "endothelial cell": "EC",  # Merge into "EC"
    "EC": "EC",  # Ensures "EC" stays as "EC"
    "MAC-M2": "MAC",
    "parietal epithelial cell": "PEC",
    "ncMON": "Mono"
    
}

# Convert categorical column to string, replace values, then convert back to categorical
adata.obs["celltype_l1"] = (
    adata.obs["celltype_l1"]
    .astype(str)  
    .replace(rename_dict)  # Apply renaming (merging)
    .astype("category")  
)

In [10]:
adata.write(os.path.join(out_dir, "NucSeq_integrated_v1_annotated_oldTransfer.h5ad"))